In [46]:
%autosave 60 

Autosaving every 60 seconds


In [47]:
#default_exp segmentation.losses_multilabel

In [48]:
import fastai; print(fastai.__version__)

1.0.58.dev0


In [49]:
#export
from fastai.vision import *
from local.segmentation.lovasz_loss import *

In [50]:
#export
_all_ = ["lovasz_softmax"]

### test

In [99]:
from local.test import *

_softmax_input = torch.cat([torch.zeros((1,1,3,3)), torch.ones((1,1,3,3))],dim=1)

_softmax_target = tensor([[[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]]])

_input.size(), _target.size()

(torch.Size([2, 4, 3, 3]), torch.Size([2, 1, 3, 3]))

In [100]:
_softmax_input.softmax(1)

tensor([[[[0.2689, 0.2689, 0.2689],
          [0.2689, 0.2689, 0.2689],
          [0.2689, 0.2689, 0.2689]],

         [[0.7311, 0.7311, 0.7311],
          [0.7311, 0.7311, 0.7311],
          [0.7311, 0.7311, 0.7311]]]])

In [102]:
_softmax_target

tensor([[[[0, 0, 0],
          [0, 1, 1],
          [0, 1, 1]]]])

### `cross_entropy`

In [69]:
# export
cross_entropy = CrossEntropyFlat(axis=1)

In [103]:
test_close(cross_entropy(_softmax_input, _softmax_target), 
           -(np.log(0.2689)*5 + np.log(0.7311)*4)/9, eps=1e-4)

### `lovasz_softmax` 

In [104]:
# random
(lovasz_softmax(_softmax_input, _softmax_target))

tensor(0.6284)

### `bce_with_logits_flat_loss`

Loss for **models that don't predict background** and use sigmoid + threshold instead of softmax/argmax.

In [105]:
#export
class LabelBinarizer(Module):    
    "converts a 2d tensor target label to 3d binary representation"
    def __init__(self, exclude_zero=True):
        self.exclude_zero = exclude_zero
        
    def forward(self, input, target):
        c, dtype = input.size(1), input.dtype
        trange = torch.arange(1 if self.exclude_zero else 0, c+1)
        trange = trange[None,...,None,None].to(target.device)
        return (target == trange).float()

In [106]:
_sigmoid_input = torch.randn(2,4,3,3)
# 0: void
_sigmoid_target = tensor([
    [[
    [1,0,0],
    [0,1,2],
    [0,3,4]
]],
      [[
    [1,0,0],
    [0,1,2],
    [0,3,4]
]]

])

_input.size(), _target.size()

(torch.Size([2, 4, 3, 3]), torch.Size([2, 1, 3, 3]))

In [85]:
label_binarizer = LabelBinarizer()
_binary_target = label_binarizer(_sigmoid_input, _sigmoid_target)
assert _binary_target.shape == _input.shape

In [18]:
#export
class _BCEWithLogitsFlatLoss(Module):
    def __init__(self, **kwargs):
        self.label_binarizer = LabelBinarizer(**kwargs)
        self.loss_fn = BCEWithLogitsFlat()
    def forward(self, input, target):
        return self.loss_fn(input, self.label_binarizer(input, target))

In [98]:
l = _BCEWithLogitsFlatLoss()
test_close(l(_input, _target), 0.83, 1e-2)

AssertionError: close:
0.73676997423172
0.83

In [96]:
# export 
bce_with_logits_flat_loss = _BCEWithLogitsFlatLoss(exclude_zero=True)

In [97]:
bce_with_logits_flat_loss.label_binarizer.exclude_zero

True

### `BCEDiceLoss`

In [40]:
#export
# https://github.com/qubvel/segmentation_models.pytorch/blob/master/segmentation_models_pytorch/utils/losses.py

def _f_score(pr, gt, beta=1, eps=1e-7, threshold=None, activation='sigmoid'):
    """
    Args:
        pr (torch.Tensor): A list of predicted elements
        gt (torch.Tensor):  A list of elements that are to be predicted
        beta (float): positive constant
        eps (float): epsilon to avoid zero division
        threshold: threshold for outputs binarization
    Returns:
        float: F score
    """

    if activation is None or activation == "none":
        activation_fn = lambda x: x
    elif activation == "sigmoid":
        activation_fn = torch.nn.Sigmoid()
    elif activation == "softmax2d":
        activation_fn = torch.nn.Softmax2d()
    else:
        raise NotImplementedError(
            "Activation implemented for sigmoid and softmax2d"
        )

    pr = activation_fn(pr)

    if threshold is not None:
        pr = (pr > threshold).float()


    tp = torch.sum(gt * pr)
    fp = torch.sum(pr) - tp
    fn = torch.sum(gt) - tp

    score = ((1 + beta ** 2) * tp + eps) \
            / ((1 + beta ** 2) * tp + beta ** 2 * fn + fp + eps)

    return score


class _DiceLoss(nn.Module):
    def __init__(self, eps=1e-7, activation='sigmoid'):
        super().__init__()
        self.activation = activation
        self.eps = eps
        self.label_binarizer = LabelBinarizer()

    def forward(self, input, target):
        target = self.label_binarizer(input, target)
        return 1 - _f_score(input, target, beta=1., eps=self.eps, 
                            threshold=None, activation=self.activation)
    
    
class _BCEDiceLoss(_DiceLoss):
    def __init__(self, eps=1e-7, activation='sigmoid'):
        super().__init__(eps, activation)
        self.bce = nn.BCEWithLogitsLoss(reduction='mean')
        self.label_binarizer = LabelBinarizer()
        
    def forward(self, input, target):
        dice = super().forward(input, target)
        target = self.label_binarizer(input, target)
        bce = self.bce(input, target)
        
        return dice + bce

In [41]:
l1, l2 = _DiceLoss(), _BCEDiceLoss()

In [42]:
_l1, _l2 = l1(_input, _target), l2(_input, _target)

In [43]:
test_close(_l2 - _l1 , 0.83, 1e-2)

### `fscore_dice_loss`

In [ ]:
#export
fscore_dice_loss  = _DiceLoss()

### `bce_dice_loss`

In [44]:
#export
bce_dice_loss = _BCEDiceLoss()

### export

In [45]:
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_script.ipynb.
Converted 02_scheduler.ipynb.
Converted 03_callbacks.ipynb.
Converted 10_segmentation_dataset.ipynb.
Converted 11_segmentation_losses_mulitlabel.ipynb.
Converted 11b_segmentation_losses_binary.ipynb.
Converted 12_segmentation_metrics.ipynb.
Converted 13_segmentation_models.ipynb.
Converted segmentation_training.ipynb.


### fin